In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

# Load the dataset
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    label_types=["detections"],
    classes=["Person"],
    max_samples=1000,  # Adjust the number of samples as needed
    dataset_dir="./open_images"
)


 100% |██████|    4.8Gb/4.8Gb [6.3m elapsed, 0s remaining, 13.7Mb/s]      


Convert Annotations to YOLO Format

In [ ]:
import os

def convert_to_yolo_format(dataset, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for sample in dataset:
        image_path = sample.filepath
        label_path = os.path.join(output_dir, os.path.basename(image_path).replace('.jpg', '.txt'))
        
        with open(label_path, 'w') as f:
            for detection in sample.detections.detections:
                if detection.label == "Person":
                    bbox = detection.bounding_box
                    class_id = 0  # 'Person' class ID
                    center_x = bbox[0] + bbox[2] / 2
                    center_y = bbox[1] + bbox[3] / 2
                    width = bbox[2]
                    height = bbox[3]
                    f.write(f"{class_id} {center_x} {center_y} {width} {height}\n")

# Convert annotations to YOLO format
convert_to_yolo_format(dataset, "./open_images/labels")


Visualize the Dataset

In [ ]:
import cv2
import matplotlib.pyplot as plt

def visualize_image(image_path, label_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    with open(label_path, 'r') as f:
        for line in f:
            class_id, center_x, center_y, width, height = map(float, line.strip().split())
            center_x, center_y, width, height = int(center_x * w), int(center_y * h), int(width * w), int(height * h)
            top_left = (center_x - width // 2, center_y - height // 2)
            bottom_right = (center_x + width // 2, center_y + height // 2)
            cv2.rectangle(image, top_left, bottom_right, (255, 0, 0), 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Visualize a few examples
images_dir = "./open_images/train/data/"
labels_dir = "./open_images/labels/"
for image_file in os.listdir(images_dir)[:5]:
    image_path = os.path.join(images_dir, image_file)
    label_path = os.path.join(labels_dir, image_file.replace('.jpg', '.txt'))
    visualize_image(image_path, label_path)
